In [8]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

# Fetch article titles

In [9]:
def fetch_and_parse_xml(url):
    # Fetch the content from URL
    response = requests.get(url)
    response.raise_for_status()  # This will raise an error if the fetch fails

    # Parse XML
    root = ET.fromstring(response.content)
    return root

def extract_information(root):
    # Define the namespace mapping
    namespaces = {
        'sa': 'https://seekingalpha.com/api/1.0',
        'media': 'http://search.yahoo.com/mrss/'
    }

    # Loop through each item in the XML
    articles = []
    for item in root.findall('.//channel/item', namespaces):
        # Extract information from each item
        title = item.find('title').text
        link = item.find('link').text
        pub_date = item.find('pubDate').text
        author_name = item.find('sa:author_name', namespaces).text
        
        # Collect article information in a dictionary
        article_info = {
            'title': title,
            'link': link,
            'published_date': pub_date,
            'author_name': author_name
        }
        articles.append(article_info)

    return articles

In [10]:
# URL of the RSS feed
url = "https://seekingalpha.com/api/sa/combined/AAPL.xml"

# Process the URL
root = fetch_and_parse_xml(url)

# Extract and print article information
articles = extract_information(root)
for article in articles:
    print(article)

{'title': 'Apple: Shiny No More - Sell Now And Keep Your Profits Thus Far', 'link': 'https://seekingalpha.com/article/4687585-apple-shiny-no-more-sell-now-and-keep-your-profits-thus-far?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024 14:00:41 -0400', 'author_name': 'DeVas Research'}
{'title': 'Apple loses No.1 spot in global smartphone market to Samsung in Q1 2024', 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024 11:04:11 -0400', 'author_name': 'Vansh Agarwal'}
{'title': "Huawei's Q1 profit soars amid rising challenge to Apple in China - report", 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024 10:01:09 -0400', 'author_name': 'Ravikash Bakolia'}
{'title': 'Apple quietly builds AI dream team, but final product remains mystery: report', 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL', 'published_date': 'Tue, 30 Apr 2024

In [13]:
def load_api_key(filepath):
    """Load and return the API key from a file."""
    with open(filepath, 'r') as file:
        api_key = file.read().strip()  # .strip() removes any leading/trailing whitespace
    return api_key

def fetch_news(api_key, query):
    url = f'https://newsapi.org/v2/everything?q=apple&apiKey='
    url += api_key
    response = requests.get(url)
    return response.json()

In [28]:
# Usage
api_key_path = "api_key.txt"
api_key = load_api_key(api_key_path)

query = 'Apple'
news_items = fetch_news(api_key, query)

# for article in news_items['articles']:
#     print(f"Title: {article['title']}")
#     print(f"Description: {article['description']}")
#     # print(f"URL: {article['url']}\n")

In [24]:
titles = []
for article in articles:
    titles.append(article['title'])
    # print(article['title'])

In [25]:
for article in news_items['articles']:
    if article['title'] != "[Removed]":
        titles.append(article['title'])
        # print(article['title'])

In [26]:
titles

['Apple: Shiny No More - Sell Now And Keep Your Profits Thus Far',
 'Apple loses No.1 spot in global smartphone market to Samsung in Q1 2024',
 "Huawei's Q1 profit soars amid rising challenge to Apple in China - report",
 'Apple quietly builds AI dream team, but final product remains mystery: report',
 "Apple's supplier list indicates China remains key manufacturing base: BofA",
 'Apple upgraded at Bernstein ahead of earnings',
 "Apple's iPad gets added to products under EU's Digital Markets Act",
 'Earnings week ahead: Amazon, Apple, Pfizer, AMD, Coca-Cola, Starbucks and more',
 'Apple faces scrutiny over iPhone sales, AI strategy in upcoming earnings call',
 'Apple, OpenAI restart talks to add generative AI to next iPhone: Bloomberg',
 'FOMC Meeting, Jobs Report, And Apple Earnings Headline Big Week',
 "Apple's removal of 'nonconsensual nude' apps is latest AI battle for big tech",
 "Huawei's new phone features updated version of advanced chip made in China - report",
 'Apple gets qu

# Sentiment Analysis

In [35]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [36]:
# Example dataset
data = {
    'title': titles
}

df = pd.DataFrame(data)

In [37]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiments
def get_sentiment(text):
    return analyzer.polarity_scores(text)

# Apply sentiment analysis
df['sentiments'] = df['title'].apply(get_sentiment)
df = pd.concat([df.drop(['sentiments'], axis=1), df['sentiments'].apply(pd.Series)], axis=1)

In [39]:
daily_sentiment = df.agg({
    'neg': 'mean',
    'neu': 'mean',
    'pos': 'mean',
    'compound': 'mean'
}).reset_index()

### Group by date if date is in the dataframe
# daily_sentiment = df.groupby('date').agg({
#     'neg': 'mean',
#     'neu': 'mean',
#     'pos': 'mean',
#     'compound': 'mean'
# }).reset_index()

print(daily_sentiment)

      index         0
0       neg  0.049707
1       neu  0.873224
2       pos  0.077069
3  compound  0.040234
